## Exercício de Pandas para Fixar oque aprendeu

#### 1) Importe os dados e verifique as primeiras linhas do dataset. Verifique também as últimas linhas do dataset.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("insurance.csv")

In [3]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


#### 2) Verifique se existe algum valor missing no dataset.

In [5]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

#### 3) Quanto de memória o dataset está consumindo?

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


A memória utilizada é de 73.3 KB

#### 4) Gere as estatísticas descritivas das colunas numéricas do dataset. Qual a mediana da coluna BMI? E a média?
Qual desses valores é maior? O que isso significa? Caso não saiba o que isso significa, passe os 2 valores ao chatGPT e pergunte a ele o que significa a média ser maior ou menor que a mediana

In [9]:
df['bmi'].median()

30.4

In [11]:
df['bmi'].mean()

np.float64(30.66339686098655)

A mediana da coluna BMI é 30.4
E a média é 30.6
A média é maior que mediana, ou seja, significa que existem valores muitos altos(outliers) puxando a média para cima. Mas neste caso está um pouco maior. então é uma leve assimetria.

#### 5) Qual a proporção de fumantes no nosso dataset? E de não fumantes?

In [15]:
df['smoker'].value_counts(normalize=True)

smoker
no     0.795217
yes    0.204783
Name: proportion, dtype: float64

79.5% da nossa base são fumantes e os restantes dos 20.5% são não fumantes

#### 6) Existe alguma diferença na cobrança para fumantes e não fumantes? Se sim, ao que você imagina que isso se deve?

In [43]:
import numpy as np

In [47]:
df["faixa_etaria"] = np.where(df['age'] < 25, '18-25',
                        np.where(df['age'] < 35, '25-35',
                        np.where(df['age'] < 45, '35-55',
                        np.where(df['age'] < 55, '45-55', '+55'))))

In [54]:
df.groupby(["smoker"])["charges"].mean()

smoker
no      8434.268298
yes    32050.231832
Name: charges, dtype: float64

In [52]:
df.groupby(["smoker","sex"])["charges"].mean()

smoker  sex   
no      female     8762.297300
        male       8087.204731
yes     female    30678.996276
        male      33042.005975
Name: charges, dtype: float64

In [51]:
df.groupby(["smoker","faixa_etaria"])["charges"].mean()

smoker  faixa_etaria
no      +55             14064.827492
        18-25            3841.102490
        25-35            5647.327950
        35-55            7545.502888
        45-55           11241.401051
yes     +55             39696.365439
        18-25           27796.537758
        25-35           28416.479738
        35-55           31366.045662
        45-55           35310.404677
Name: charges, dtype: float64

Sim, existe uma diferença significativa na cobrança entre fumantes e não fumantes.
Os fumantes tendem a gastar muito mais do que os não fumantes.

Ao segmentar por gênero, observa-se que entre os não fumantes, as mulheres têm uma média de cobrança ligeiramente maior do que os homens. Já entre os fumantes, ocorre o inverso: os homens apresentam uma média de cobrança mais alta que as mulheres.

Quando analisamos por faixa etária, percebe-se que, em ambos os grupos, quanto maior a idade, maior o valor médio das cobranças.
No entanto, entre os não fumantes, o aumento dos gastos com a idade é mais acentuado, ou seja, as diferenças entre as faixas etárias são maiores.
Já entre os fumantes, embora também haja crescimento com a idade, a variação é mais suave.

#### 7) Qual a média de BMI para fumantes? Ela é muito diferente da observada para fumantes? Se sim, por qual motivo você acha que isso ocorre?


In [53]:
df.groupby(["smoker"])["bmi"].mean()

smoker
no     30.651795
yes    30.708449
Name: bmi, dtype: float64

Não há quase nenhuma diferença entre a média de bmi entre os fumantes e não fumantes.

#### 8) Existe alguma diferença de cobrança para quem tem filhos? Identifique se ter filhos parece acarretar numa cobrança maior olhando a média de charges para quem tem 0 filhos e a média para quem tem 1 ou mais filhos.

In [59]:
df.groupby(["children"])["charges"].mean()

children
0    12365.975602
1    12731.171832
2    15073.563734
3    15355.318367
4    13850.656311
5     8786.035247
Name: charges, dtype: float64

In [57]:
df["filhos"] = np.where(df['children'] == 0, '0', '1 ou mais')

In [58]:
df.groupby(["filhos"])["charges"].mean()

filhos
0            12365.975602
1 ou mais    13949.941093
Name: charges, dtype: float64

Sim há uma diferença na cobrança entre ter filhos ou não. quem tem filhos acabam tendo mais gastos comparados com aqueles que não tem filhos.

#### 9) Para o homem, o BMI de 20 a 24,9 é considerado normal. Acima disso, é considerado acima do peso, abaixo é considerado abaixo do peso. Crie uma nova coluna, chamada "bmi_male" que receberá o texto "abaixo do normal" para BMI abaixo de 20, "normal" para BMI de 20 a 24,9 e "acima do normal" para BMI acima de 24,9.

In [66]:
df['bmi_male'] = np.where((df['bmi'] < 20) & (df['sex'] == 'male'), 'abaixo do normal',
                    np.where((df['bmi'] <=24.9) & (df['sex'] == 'male'), 'normal',
                    np.where((df['bmi'] > 24.9) & (df['sex'] == 'male'), 'acima do normal','n/a')))

In [77]:
df[df['bmi_male'] != 'n/a']['bmi_male'].value_counts(normalize=True)

bmi_male
acima do normal     0.831361
normal              0.140533
abaixo do normal    0.028107
Name: proportion, dtype: float64

podemos perceber que 83% dos homens estão acima do peso

In [80]:
df_filtrado = df[df['bmi_male'] != 'n/a']
df_filtrado.groupby('bmi_male')['charges'].mean()

bmi_male
abaixo do normal     8548.158665
acima do normal     14864.695657
normal               9667.261289
Name: charges, dtype: float64

Podemos perceber que pessoas acima do normal gastam mais tambem.

#### Utilizando o dataset de incêndios florestais no Brasil, que você pode baixar clicando aqui(https://www.kaggle.com/datasets/gustavomodelli/forest-fires-in-brazil), responda:

#### 11) Carregue o dataset e crie uma mensagem ao leitor do notebook falando quantas linhas e colunas possui o DataFrame.

In [84]:
df = pd.read_csv("amazon.csv", encoding='latin1')

In [85]:
df.head()

,year,state,month,number,date
0,1998,Acre,Janeiro,0.0,1998-01-01
1,1999,Acre,Janeiro,0.0,1999-01-01
2,2000,Acre,Janeiro,0.0,2000-01-01
3,2001,Acre,Janeiro,0.0,2001-01-01
4,2002,Acre,Janeiro,0.0,2002-01-01


In [89]:
df.shape
print(f"Total linhas:{df.shape[0]}, Total colunas: {df.shape[1]}")

Total linhas:6454, Total colunas: 5


#### 12) Utilize loc e iloc para exibir da linha 100 até a linha 120 da segunda, terceira e quarta coluna. Qual a diferença das duas sintaxes feitas?

In [97]:
df.iloc[100:121, 1:4]

,state,month,number
100,Acre,Junho,3.0
101,Acre,Junho,0.0
102,Acre,Junho,1.0
103,Acre,Junho,1.0
104,Acre,Junho,0.0
105,Acre,Junho,0.0
106,Acre,Junho,5.0
107,Acre,Junho,27.0
108,Acre,Junho,1.0
109,Acre,Junho,4.0


In [99]:
df.loc[100:121, df.columns[1:4]]

,state,month,number
100,Acre,Junho,3.0
101,Acre,Junho,0.0
102,Acre,Junho,1.0
103,Acre,Junho,1.0
104,Acre,Junho,0.0
105,Acre,Junho,0.0
106,Acre,Junho,5.0
107,Acre,Junho,27.0
108,Acre,Junho,1.0
109,Acre,Junho,4.0


#### 13) Qual o total de incêndios que ocorreram no Brasil no ano de 2010? E se considerarmos somente o estado do Mato Grosso do Sul?

In [110]:
total_incendio_2010 = df.loc[df['year']==2010,'number'].sum()

In [107]:
df['state'].unique()

array(['Acre', 'Alagoas', 'Amapa', 'Amazonas', 'Bahia', 'Ceara',
       'Distrito Federal', 'Espirito Santo', 'Goias', 'Maranhao',
       'Mato Grosso', 'Minas Gerais', 'Pará', 'Paraiba', 'Pernambuco',
       'Piau', 'Rio', 'Rondonia', 'Roraima', 'Santa Catarina',
       'Sao Paulo', 'Sergipe', 'Tocantins'], dtype=object)

In [111]:
total_incendio_2010_mato_grosso = df.loc[(df['year']==2010) & (df['state']=='Mato Grosso'), 'number'].sum()

In [113]:
print(f"total incendia de 2010 foi {total_incendio_2010} incendios e de mato grosso neste ano foi de {total_incendio_2010_mato_grosso} incendios")

total incendia de 2010 foi 37037.449 incendios e de mato grosso neste ano foi de 6252.445 incendios


#### 14) Qual a média de incêndios por mês no estado do Acre? E o desvio padrão?

In [115]:
df.loc[df['state'] == 'Acre'].groupby('month')['number'].mean()

month
Abril          2.050000
Agosto       319.651700
Dezembro       2.473684
Fevereiro      0.950000
Janeiro        2.000000
Julho        114.800000
Junho         12.850000
Maio           4.050000
Março          1.150000
Novembro      35.400000
Outubro      362.325400
Setembro      65.624400
Name: number, dtype: float64

In [116]:
df.loc[df['state'] == 'Acre'].groupby('month')['number'].std()

month
Abril          4.334379
Agosto       336.205557
Dezembro       2.893833
Fevereiro      1.637553
Janeiro        4.142209
Julho        153.931837
Junho         21.976722
Maio           5.114222
Março          2.497894
Novembro      36.864474
Outubro      265.668310
Setembro     139.748975
Name: number, dtype: float64

#### 15) Qual estado possui a maior média de incêndios por mês? E qual possui a menor?

In [132]:
grupo = df.groupby(['state', 'month'])['number'].mean()
maior = grupo.idxmax()
menor = grupo.idxmin()

valor_maximo = grupo.max()
valor_minimo = grupo.min()
print(f"O estado de {maior[0]} possuí maior média de incêndio de {valor_maximo} incêndio no mês de {maior[1]}.")
print(f"O estado de {menor[0]} possuí menor média de incêndio de {valor_minimo} incêndio no mês de {menor[1]}.")

O estado de Sao Paulo possuí maior média de incêndio de 495.12725 incêndio no mês de Agosto.
O estado de Sergipe possuí menor média de incêndio de 0.05 incêndio no mês de Julho.


#### 16) Crie uma coluna chamada regiao_sudeste e marque com 1 para quando a observação pertencer a um destes estados e 0, caso contrário. Faça o mesmo para a região centro-oeste. Qual a diferença no total de incêndios dessas regiões? Qual região possui mais incêndios? Consulte a internet para entender por qual motivo uma das regiões possui mais incêndio que a outra.

In [136]:
df['regiao_sudeste'] = df['state'].map(lambda x: 1 if x in ['Espirito Santo', 'Minas Gerais', 'Rio','Sao Paulo'] else 0)

In [141]:
df[df['regiao_sudeste'] == 1]['state'].unique()

array(['Espirito Santo', 'Minas Gerais', 'Rio', 'Sao Paulo'], dtype=object)

In [143]:
df.groupby('state')['regiao_sudeste'].mean()

state
Acre                0.0
Alagoas             0.0
Amapa               0.0
Amazonas            0.0
Bahia               0.0
Ceara               0.0
Distrito Federal    0.0
Espirito Santo      1.0
Goias               0.0
Maranhao            0.0
Mato Grosso         0.0
Minas Gerais        1.0
Paraiba             0.0
Pará                0.0
Pernambuco          0.0
Piau                0.0
Rio                 1.0
Rondonia            0.0
Roraima             0.0
Santa Catarina      0.0
Sao Paulo           1.0
Sergipe             0.0
Tocantins           0.0
Name: regiao_sudeste, dtype: float64

In [148]:
df['regiao_centro_oeste'] = df['state'].map(lambda x: 1 if x in [ 'Distrito Federal', 'Goias', 'Mato Grosso'] else 0)

In [154]:
df.drop(columns = 'centro_oeste', inplace =True)

In [155]:
df.drop(columns = 'egiao_centro_oeste', inplace =True)

In [156]:
df[df['regiao_centro_oeste'] == 1]['state'].unique()

array(['Distrito Federal', 'Goias', 'Mato Grosso'], dtype=object)

In [157]:
df.groupby('state')['regiao_centro_oeste'].mean()

state
Acre                0.0
Alagoas             0.0
Amapa               0.0
Amazonas            0.0
Bahia               0.0
Ceara               0.0
Distrito Federal    1.0
Espirito Santo      0.0
Goias               1.0
Maranhao            0.0
Mato Grosso         1.0
Minas Gerais        0.0
Paraiba             0.0
Pará                0.0
Pernambuco          0.0
Piau                0.0
Rio                 0.0
Rondonia            0.0
Roraima             0.0
Santa Catarina      0.0
Sao Paulo           0.0
Sergipe             0.0
Tocantins           0.0
Name: regiao_centro_oeste, dtype: float64

In [165]:
incendio_regiao_sudeste = df[df['regiao_sudeste'] == 1]['number'].sum()
print(incendio_regiao_sudeste)
incendio_regiao_centro_oeste = df[df['regiao_centro_oeste'] == 1]['number'].sum()
print(incendio_regiao_centro_oeste)
if incendio_regiao_sudeste > incendio_regiao_centro_oeste:
    print(f"Região Sudeste tem mais incêndio que Região Centro-Oeste, total de {incendio_regiao_sudeste} incêndios")
else:
    print(f"Região Centro-Oeste tem mais incêndio que Região Sudeste, total de {incendio_regiao_centro_oeste} incêndios")

140303.321
137502.54799999998
Região Sudeste tem mais incêndio que Região Centro-Oeste, total de 140303.321 incêndios
